In [ ]:
import os
import glob
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationRetrievalChain
from langchain_core.callbacks import StdOutCallbackHandler
from langchain_community.embeddings import OllamaEmbeddings
import gradio as gr

folders = glob.glob('knowledge-base/*')

text_loaders_kwargs={'encoding':'utf-8'}

documents=[]

for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob='**/*.md', loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
    folder_docs = loader.load()
    for doc in folder_docs:
        doc.metadata['doc_type'] = doc_type
        documents.append(doc)

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings(base_url='http://localhost:11434/v1', api_key='ollama')

vector_store = FAISS.from_documents(chunk,embeddings)

llm = ChatOpenAI(model="llama3.2", api_key="ollama", temperature=0.7)

memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

retriever = vector_store.as_retriever(search_kwargs={'k':25})

conversation_chain = ConversationRetrievalChain.fromm_llm(llm=llm, retriever=retriever, memory=memory, callbacks=[StdOutCallbackHandler()])

def chat(message, history):
    result = conversation_chain.invoke({'question': message})
    return result['answer']

gr.ChatInterface(fn=chat, type='messages').launch()    



